<a href="https://colab.research.google.com/github/cdbm/airbnb-price-rediction-optimizer/blob/main/airbnb_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np 
import pandas as pd

# Database pre processing

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/cdbm/airbnb-price-rediction-optimizer/main/AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [5]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [6]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [7]:
df = df.drop('name', axis=1)
df = df.drop('host_name', axis=1)
df = df.drop('last_review', axis=1)

In [8]:
df = df[df.price <= 500]

In [9]:
# df.loc[df['price'] <= 50, 'ranges'] = '0_TO_50'
df.loc[df['price'] <= 100, 'ranges'] = '0_TO_100'
df.loc[(df['price'] > 100) & (df['price'] <= 200), 'ranges'] = '101_TO_200'
# df.loc[(df['price'] > 150) & (df['price'] <= 200), 'ranges'] = '151_TO_200'
df.loc[(df['price'] > 200) & (df['price'] <= 300), 'ranges'] = '201_TO_300'
#df.loc[(df['price'] > 250) & (df['price'] <= 300), 'ranges'] = '251_TO_300'
df.loc[(df['price'] > 300) & (df['price'] <= 400), 'ranges'] = '301_TO_400'
#df.loc[(df['price'] > 350) & (df['price'] <= 400), 'ranges'] = '351_TO_400'
df.loc[(df['price'] > 400) & (df['price'] <= 500), 'ranges'] = '401_TO_500'
#df.loc[(df['price'] > 450) & (df['price'] <= 500), 'ranges'] = '551_TO_500'

In [10]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')
df['neighbourhood'] = df['neighbourhood'].astype('category')
df['room_type'] = df['room_type'].astype('category')

In [11]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [12]:
df['ranges'] = df['ranges'].astype('category')

In [13]:
df['reviews_per_month'].fillna(0, inplace=True) 

In [14]:
df = df.drop('price', axis=1)


In [15]:
df.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,2539,2787,1,107,40.64749,-73.97237,1,1,9,0.21,6,365,101_TO_200
1,2595,2845,2,126,40.75362,-73.98377,0,1,45,0.38,2,355,201_TO_300
2,3647,4632,2,93,40.80902,-73.94190,1,3,0,0.00,1,365,101_TO_200
3,3831,4869,1,41,40.68514,-73.95976,0,1,270,4.64,1,194,0_TO_100
4,5022,7192,2,61,40.79851,-73.94399,0,10,9,0.10,1,0,0_TO_100


In [16]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:10000]
df.describe()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000
mean,1.919970e+07,6.784959e+07,1.674800,106.927400,40.729377,-73.951170,0.509000,7.01150,22.520100,1.096590,7.154800,110.08350
std,1.094391e+07,7.888906e+07,0.742631,68.213385,0.054883,0.046102,0.544563,18.18126,43.390262,1.600581,32.983803,130.37465
min,5.099000e+03,2.438000e+03,0.000000,0.000000,40.539870,-74.172580,0.000000,1.00000,0.000000,0.000000,1.000000,0.00000
25%,9.682449e+06,7.832395e+06,1.000000,51.000000,40.689765,-73.982450,0.000000,1.00000,1.000000,0.040000,1.000000,0.00000
50%,2.002976e+07,3.042066e+07,2.000000,93.000000,40.723215,-73.955470,0.000000,3.00000,5.000000,0.380000,1.000000,41.00000
75%,2.927896e+07,1.074549e+08,2.000000,178.000000,40.763823,-73.934938,1.000000,5.00000,23.000000,1.550000,2.000000,218.00000
max,3.648436e+07,2.743115e+08,4.000000,218.000000,40.904060,-73.716900,2.000000,500.00000,576.000000,20.940000,327.000000,365.00000


In [17]:
df

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,21118650,145187951,1,13,40.68242,-73.95754,1,3,18,1.04,2,0,0_TO_100
1,32522010,51810229,3,165,40.69903,-73.90961,0,28,0,0.00,1,197,101_TO_200
2,6090,11975,2,208,40.73530,-74.00525,0,90,27,0.22,1,0,101_TO_200
3,29081056,48954003,0,148,40.88453,-73.86273,1,2,20,2.20,1,188,0_TO_100
4,32070946,48146336,2,94,40.76318,-73.99335,0,30,2,0.50,20,263,101_TO_200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,21411685,35662919,1,213,40.71023,-73.95827,1,2,1,0.05,1,0,0_TO_100
9996,3710436,318729,1,13,40.67862,-73.94687,0,3,118,2.12,1,23,101_TO_200
9997,32808879,219517861,2,136,40.74851,-73.97431,0,29,0,0.00,327,341,101_TO_200
9998,34258598,195863013,2,201,40.80249,-73.96729,1,3,3,2.05,1,2,0_TO_100


In [18]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [19]:
  features = df.drop('ranges',axis=1)
  labels = df['ranges']

In [20]:
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [21]:
sc_X = StandardScaler()
feature_train=sc_X.fit_transform(feature_train)
feature_test=sc_X.fit_transform(feature_test)

# Genetic Algorithm

In [22]:
import random

In [23]:
POPULATION_SIZE = 30
GENERATIONS = 100
CHILDREN_RATE = 3

In [24]:
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs','sgd', 'adam']
learning_rate = ['constant', 'invscaling', 'adaptive']

In [25]:
def generate_individue():
  individue = {
      'activation':  activation[random.randint(0, 3)],
      'solver': solver[random.randint(0,2)],
      'learning_rate': learning_rate[random.randint(0,2)],
      'hidden_layer_sizes' : (random.randint(100, 200), random.randint(50, 100) ,random.randint(10, 50)),
      'max_iter' : random.randint(100, 250),
      'fitness': 0  
  }

  individue['fitness'] = calculate_fitness(individue)
  return individue

In [26]:
def sortByFitness(e):
  return e['fitness']

In [27]:
def initiate_population(population_size):
  population = [[]] * population_size
  population = [generate_individue() for i in range(0, population_size)]
  population.sort(key=sortByFitness, reverse=True)
  return population

In [28]:
def choose_parents(population):
  father = random.randint(0, POPULATION_SIZE - 1)
  mother = random.randint(0, POPULATION_SIZE - 1)
  return [population[father], population[mother]]

In [29]:
def choose_candidates(population):
  candidatesIndex = []
  for x in range(0, round(POPULATION_SIZE * 0.3) ):
    index = random.randint(0, POPULATION_SIZE - 1)
    while index in candidatesIndex:
      index = random.randint(0, POPULATION_SIZE - 1)
    candidatesIndex.append(index)

  candidates = []
  for x in candidatesIndex:
    candidates.append(population[x])
  return candidates   

def choose_parents_by_roullete(population):
  population.sort(key=sortByFitness, reverse=True)
  candidates = population[:round(POPULATION_SIZE / 2)]
  probabilities = [];
  sumOfFitness = 0;
  previousProbabilities = 0;
  parents = [];

  for i in candidates:
    sumOfFitness = sumOfFitness + i['fitness']

  candidates.sort(key=sortByFitness)

 
  for cand in candidates:
    prob = previousProbabilities + (cand['fitness'] / sumOfFitness);
    previousProbabilities = prob;
    probabilities.append(prob);
  
  while len(parents)  < 2:
    number = random.randint(1, 10) /10
    found = False
    for y in range(0, len(probabilities)):
      if (number <= probabilities[y]) & (not found):
        parents.append(candidates[y])
        found = True
  
  return parents

In [30]:
def mutate_activation(individue):
  new_attribute = activation[random.randint(0,3)] 
  while new_attribute == individue['activation']:
    new_attribute = activation[random.randint(0,3)] 
  individue['activation'] = new_attribute
  return individue

def mutate_learning_rate(individue):
  new_attribute = learning_rate[random.randint(0,2)] 
  while new_attribute == individue['learning_rate']:
    new_attribute = learning_rate[random.randint(0,2)] 
  individue['learning_rate'] = new_attribute
  return individue

def mutate_solver(individue):
  new_attribute = solver[random.randint(0,2)] 
  while new_attribute == individue['solver']:
    new_attribute = solver[random.randint(0,2)] 
  individue['solver'] = new_attribute
  return individue

def mutate(individue):
  mutations = [mutate_activation, mutate_learning_rate, mutate_solver]
  prob = random.randint(0, 10)
  if prob >= 6:
    mutation1 = random.randint(0,2)
    mutation2 = random.randint(0,2)

    individue = mutations[mutation1](individue)
    individue = mutations[mutation2](individue)

  return individue

In [31]:
def make_children(population):
  parents = choose_parents_by_roullete(population)
  
  for x in range(0, 2):
    child = {'activation': parents[random.randint(0,1)]['activation'],
            'hidden_layer_sizes': (parents[random.randint(0,1)]['hidden_layer_sizes'][0], parents[random.randint(0,1)]['hidden_layer_sizes'][1], parents[random.randint(0,1)]['hidden_layer_sizes'][2]),
            'learning_rate': parents[random.randint(0,1)]['learning_rate'],
            'max_iter': parents[random.randint(0,1)]['max_iter'],
            'solver': parents[random.randint(0,1)]['solver'],
            'fitness': 0}
  
    child = mutate(child)
    child['fitness'] = calculate_fitness(child)
    population.append(child)
  
  population.sort(key=sortByFitness, reverse=True)
  return population[:POPULATION_SIZE]

In [32]:
def calculate_fitness(individue):
  model_mlp = MLPClassifier(activation=individue['activation'],
                            hidden_layer_sizes=individue['hidden_layer_sizes'],
                            learning_rate=individue['learning_rate'],
                            max_iter=individue['max_iter'],
                            solver=individue['solver'])
  model_mlp.fit(feature_train, label_train) 

  prediction = model_mlp.predict(feature_test) 
  return accuracy_score(label_test, prediction)
  

Main


In [33]:
population = initiate_population(POPULATION_SIZE)
population.sort(key=sortByFitness, reverse=True)
print(population)
for x in range(0, GENERATIONS):
  print("generation: ")
  print(x)
  population = make_children(population)
  print(population)

print("Better was: ") 
print(population[0])
print("With a accuracy of: ")
print(population[0]['fitness'])

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (214) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (161) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571:

[{'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (118, 90, 31), 'max_iter': 237, 'fitness': 0.7085}, {'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (188, 99, 19), 'max_iter': 184, 'fitness': 0.7025}, {'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (158, 82, 14), 'max_iter': 192, 'fitness': 0.7015}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (199, 67, 41), 'max_iter': 187, 'fitness': 0.7015}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (124, 83, 42), 'max_iter': 250, 'fitness': 0.701}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (132, 71, 14), 'max_iter': 136, 'fitness': 0.701}, {'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (112, 66, 34), 'ma

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (240) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (240) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (118, 90, 31), 'max_iter': 237, 'fitness': 0.7085}, {'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (188, 99, 19), 'max_iter': 184, 'fitness': 0.7025}, {'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (158, 82, 14), 'max_iter': 192, 'fitness': 0.7015}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (199, 67, 41), 'max_iter': 187, 'fitness': 0.7015}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (124, 83, 42), 'max_iter': 250, 'fitness': 0.701}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (132, 71, 14), 'max_iter': 136, 'fitness': 0.701}, {'activation': 'logistic', 'hidden_layer_sizes': (120, 51, 19), 'learning_rate': 'constant', 'max_iter': 240,

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (187) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[{'activation': 'logistic', 'hidden_layer_sizes': (120, 99, 31), 'learning_rate': 'invscaling', 'max_iter': 187, 'solver': 'lbfgs', 'fitness': 0.7125}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (118, 90, 31), 'max_iter': 237, 'fitness': 0.7085}, {'activation': 'logistic', 'hidden_layer_sizes': (124, 99, 14), 'learning_rate': 'invscaling', 'max_iter': 187, 'solver': 'lbfgs', 'fitness': 0.7085}, {'activation': 'logistic', 'hidden_layer_sizes': (158, 51, 14), 'learning_rate': 'constant', 'max_iter': 250, 'solver': 'adam', 'fitness': 0.708}, {'activation': 'logistic', 'hidden_layer_sizes': (132, 83, 14), 'learning_rate': 'constant', 'max_iter': 136, 'solver': 'lbfgs', 'fitness': 0.7075}, {'activation': 'relu', 'hidden_layer_sizes': (124, 65, 14), 'learning_rate': 'constant', 'max_iter': 250, 'solver': 'sgd', 'fitness': 0.707}, {'activation': 'logistic', 'hidden_layer_sizes': (124, 99, 31), 'learning_rate': 'invscaling', 'max_iter': 187

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'logistic', 'hidden_layer_sizes': (120, 99, 31), 'learning_rate': 'invscaling', 'max_iter': 187, 'solver': 'lbfgs', 'fitness': 0.7125}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (118, 90, 31), 'max_iter': 237, 'fitness': 0.7085}, {'activation': 'logistic', 'hidden_layer_sizes': (124, 99, 14), 'learning_rate': 'invscaling', 'max_iter': 187, 'solver': 'lbfgs', 'fitness': 0.7085}, {'activation': 'logistic', 'hidden_layer_sizes': (158, 51, 14), 'learning_rate': 'constant', 'max_iter': 250, 'solver': 'adam', 'fitness': 0.708}, {'activation': 'logistic', 'hidden_layer_sizes': (132, 83, 14), 'learning_rate': 'constant', 'max_iter': 136, 'solver': 'lbfgs', 'fitness': 0.7075}, {'activation': 'relu', 'hidden_layer_sizes': (124, 65, 14), 'learning_rate': 'constant', 'max_iter': 250, 'solver': 'sgd', 'fitness': 0.707}, {'activation': 'relu', 'hidden_layer_sizes': (124, 65, 14), 'learning_rate': 'constant', 'max_iter': 250, 'sol

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
